- CNN policy ?
- grid search for HP tuning (OK)
- Increasingly difficult Environment
- Positive reward for populating increasingly "deep" blending tanks ?
- RL for chem sched paper (https://arxiv.org/pdf/2203.00636)
- Masking (https://sb3-contrib.readthedocs.io/en/master/modules/ppo_mask.html, https://arxiv.org/pdf/2006.14171)
    - Adding binary decision variables ?g  
    - Requires discrete action space (only integer flows -> treated as categories ?)
    - masking: disable incoming flows (resp. outgoing flows) for tanks at UB inv limit (resp. LB inv. limit), disable selling/buying when available = 0
    - multiple envs with multiple agents ? (MARL, https://arxiv.org/pdf/2103.01955)
        - Predict successive pipelines ("source > blend" then "blend > blend" (as many as required) then "blend > demand")
        - Each agent has access to the whole state
        - Action mask is derived from the previous agent's actions (0 if inventory at bounds or incoming flow already reserved, else 1)
        - https://github.com/Rohan138/marl-baselines3/blob/main/marl_baselines3/independent_ppo.py
- Safe RL: (https://proceedings.mlr.press/v119/wachi20a/wachi20a.pdf)
    - "Unsafe state" ? > Do not enforce constraints strictly, instead opt for early episode termination to show which states are unsafe ? 
    - Implementations:
        - https://pypi.org/project/fast-safe-rl/#description (Policy optimizers)
        - https://github.com/PKU-Alignment/safety-gymnasium/tree/main/safety_gymnasium (environments; "cost" ?)


1. Try other learning rates/CNN policies
2. Implement Masking with single agent
3. Try other ways to tell the model what are illegal/unsafe states (safe RL)
4. Try multiple agents

-----------------------

- Masking: Discretization of action space is too slow/might not work -> Need to implement masking for continuous action space
- Recurrent policy makes the most sense ? (window of demand forecasts)
- https://www.reddit.com/r/reinforcementlearning/comments/17l5b47/invalid_action_masking_when_action_space_is/
    - Suggestion of autoregressive model for having constraints respected: one predicted action is input to a second model
    - Suggestion of editing the distribution in such a way that the constraint is respected
- https://www.sciencedirect.com/science/article/pii/S0098135420301599
    - Choice of ELU activation ?
    - Choice of NN size ?
    - "The feature engineering in the net inventory means the network does not have to learn these relationships itself, which did help speed training." ?
- Simplify the problem (remove tanks 5 to 8), find the optimal solution with Gurobi

- remove all constraints except in/out
- https://arxiv.org/pdf/1711.11157
- https://arxiv.org/pdf/2111.01564
- Softmax with large coef to produce action mask
- Graph convolution NN instead of RNN ?
    - https://pytorch-geometric.readthedocs.io/en/latest/
    - Graph rep. learning - William L Hamilton

In [27]:
# import gymnasium as gym
import json
import numpy as np
import torch as th
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from envs import BlendEnv, flatten_and_track_mappings, reconstruct_dict
from models import CustomRNNPolicy

In [28]:
action_sample = {
    'source_blend':{
        's1': {'j1':1, 'j2':1, 'j3':1, 'j4':0}, # From s1 to b1, from s1 to b2 etc...
        's2': {'j1':1, 'j2':1, 'j3':0, 'j4':1},
    },
    
    # 'source_demand':{
    #     's1': {},
    #     's2': {}
    # },
    
    'blend_blend':{
        'j1': {'j5':1, 'j6':0, 'j7':0, 'j8':0},
        'j2': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j3': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j4': {'j5':0, 'j6':0, 'j7':0, 'j8':0},
        'j5': {},
        'j6': {},
        'j7': {},
        'j8': {}
    },
    
    'blend_demand':{
        'j1': {},
        'j2': {},
        'j3': {},
        'j4': {},
        'j5': {'p1':1, 'p2':0},
        'j6': {'p1':1, 'p2':2},
        'j7': {'p1':1, 'p2':2},
        'j8': {'p1':1, 'p2':2}
    },
    
    "tau": {"s1": 10, "s2": 10},
    
    "delta": {"p1": 0, "p2": 0}
}
action_sample_flat, _ = flatten_and_track_mappings(action_sample)

In [29]:
def lr_scheduler(p):
    if p > 0.9:
        return 1e-2
    if p > 0.75:
        return 4e-3
    if p > 0.25:
        return 1e-3
    else:
        return 2e-4

In [30]:
def lr_scheduler2(p):
    if p > 0.9:
        return 1e-2
    if p > 0.6:
        return 5e-3
    else:
        return 2e-3

In [31]:
env = BlendEnv(v = False, D=50)
env = Monitor(env)

In [32]:
policy_kwargs = dict(
    net_arch=[dict(pi=[128]*5, vf=[128]*5)]
)

In [33]:
model = PPO("MlpPolicy", env, tensorboard_log="./logs", clip_range=1e4, learning_rate=lr_scheduler2, policy_kwargs=policy_kwargs, ent_coef=0.1)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [12]:
# model = PPO(CustomRNNPolicy, env, tensorboard_log="./logs", clip_range=0.4, learning_rate=lr_scheduler, policy_kwargs=policy_kwargs, ent_coef=0)

In [34]:
import datetime
model_name = f"model_{datetime.datetime.now().strftime('%m%d-%H%M')}_ent_{model.ent_coef}_gam_{model.gamma}_clip_{model.clip_range(0)}_{int(env.M)}_{int(env.Z)}_{int(env.P)}"
model_name

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.M to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.M` for environment variables or `env.get_wrapper_attr('M')` that will search the reminding wrappers.
  logger.warn(
c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.Z to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.Z` for environment variables or `env.get_wrapper_attr('Z')` that will search the reminding wrappers.
  logger.warn(
c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL\blendv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variable

'model_0530-2315_ent_0.1_gam_0.99_clip_10000.0_1000_10000_100'

In [56]:
# model.set_parameters('model_0527-2004_ent_0.0_gam_0.99_clip_0.4_1000_10000_100')

In [35]:
model.learn(total_timesteps=1000000, progress_bar=False)

KeyboardInterrupt: 

In [36]:
model.save(model_name)

In [78]:
# model.save(f"./models/test_newmodel")

In [8]:
model.set_parameters('model_0530-2312_ent_0.5_gam_0.99_clip_0.5_1000_10000_100')

In [37]:
envv = BlendEnv(v = True, D = 50)

In [38]:
# 0 (only once per episode)
# Get first state (we know what it looks like)
episode_rewards = []
obs = envv.reset()
obs, obs_dict = obs

In [39]:
# 1
# Get first action
action, _ = model.predict(obs, deterministic=True)

In [40]:
# 2
# Visualize action
reconstruct_dict(action, envv.mapping_act)

{'source_blend': {'s1': {'j1': 28.647383,
   'j2': 0.0,
   'j3': 42.656128,
   'j4': 2.0147858},
  's2': {'j1': 0.0, 'j2': 0.0, 'j3': 6.1936836, 'j4': 50.0}},
 'blend_blend': {'j1': {'j5': 30.696165,
   'j6': 31.245146,
   'j7': 50.0,
   'j8': 0.0},
  'j2': {'j5': 0.0, 'j6': 0.0, 'j7': 1.105059, 'j8': 0.0},
  'j3': {'j5': 32.77132, 'j6': 20.790638, 'j7': 22.801126, 'j8': 0.90838027},
  'j4': {'j5': 0.0, 'j6': 11.395435, 'j7': 0.0, 'j8': 0.0}},
 'blend_demand': {'j5': {'p1': 0.0, 'p2': 0.0},
  'j6': {'p1': 49.36426, 'p2': 46.820507},
  'j7': {'p1': 24.837915, 'p2': 0.0},
  'j8': {'p1': 0.0, 'p2': 36.694725}},
 'tau': {'s1': 31.190184, 's2': 29.173275},
 'delta': {'p1': 0.0, 'p2': 50.0}}

In [41]:
# 3
# Step once: get 2nd action
obs, reward, done, term, _ = envv.step(action)

[PEN] t1; s1:			bought too much (more than supply)
[PEN] t1; p2:			sold too much (more than demand)
[PEN] t1; s1:			bought too little (resulting amount less than source tank LB)
Increased reward by 0 through tank population in s1
[PEN] t1; s2:			bought too little (resulting amount less than source tank LB)
Increased reward by 0 through tank population in s2
[PEN] t1; j1:			In and out flow both non-zero (in: 28.65, out: 111.94)
[PEN] t1; j2:			inventory OOB (resulting amount less than blending tank LB)
[PEN] t1; j3:			In and out flow both non-zero (in: 48.85, out: 77.27)
[PEN] t1; j4:			In and out flow both non-zero (in: 52.01, out: 11.4)
Increased reward by 0 through tank population in j5
[PEN] t1; j6:			inventory OOB (resulting amount less than blending tank LB)
[PEN] t1; j7:			inventory OOB (resulting amount less than blending tank LB)
[PEN] t1; j8:			inventory OOB (resulting amount less than blending tank LB)
Increased reward by 0 through tank population in p1
Increased reward by 0 

In [42]:
# 4
# Visualize new state
reconstruct_dict(obs, envv.mapping_obs)
# Go back to step 1 now

{'sources': {'s1': -63.3183, 's2': -26.193684},
 'blenders': {'j1': 0.0,
  'j2': 0.0,
  'j3': 0.0,
  'j4': 0.0,
  'j5': 0.0,
  'j6': 0.0,
  'j7': 0.0,
  'j8': 0.0},
 'demands': {'p1': 0.0, 'p2': 0.0},
 'properties': {'j1': {'q1': 0.0},
  'j2': {'q1': 0.0},
  'j3': {'q1': 0.0},
  'j4': {'q1': 0.0},
  'j5': {'q1': 0.0},
  'j6': {'q1': 0.0},
  'j7': {'q1': 0.0},
  'j8': {'q1': 0.0}},
 'sources_avail_next_0': {'s1': 10.0, 's2': 30.0},
 'demands_avail_next_0': {'p1': 0.0, 'p2': 0.0},
 'sources_avail_next_1': {'s1': 10.0, 's2': 30.0},
 'demands_avail_next_1': {'p1': 15.0, 'p2': 15.0},
 'sources_avail_next_2': {'s1': 0.0, 's2': 0.0},
 'demands_avail_next_2': {'p1': 15.0, 'p2': 15.0},
 'sources_avail_next_3': {'s1': 0.0, 's2': 0.0},
 'demands_avail_next_3': {'p1': 15.0, 'p2': 15.0},
 'sources_avail_next_4': {'s1': 0.0, 's2': 0.0},
 'demands_avail_next_4': {'p1': 15.0, 'p2': 15.0},
 'sources_avail_next_5': {'s1': 0.0, 's2': 0.0},
 'demands_avail_next_5': {'p1': 0.0, 'p2': 0.0}}

In [42]:
reward

-874.4678911447525

In [ ]:
# End of episode
episode_rewards.append(reward)

In [16]:
with open("./connections_sample.json" ,"r") as f:
    connections_s = f.readline()
connections = json.loads(connections_s)
connections

{'source_blend': {'s1': ['j1', 'j2', 'j3', 'j4'],
  's2': ['j1', 'j2', 'j3', 'j4']},
 'blend_blend': {'j1': ['j5', 'j6', 'j7', 'j8'],
  'j2': ['j5', 'j6', 'j7', 'j8'],
  'j3': ['j5', 'j6', 'j7', 'j8'],
  'j4': ['j5', 'j6', 'j7', 'j8'],
  'j5': [],
  'j6': [],
  'j7': [],
  'j8': []},
 'blend_demand': {'j1': [],
  'j2': [],
  'j3': [],
  'j4': [],
  'j5': ['p1', 'p2'],
  'j6': ['p1', 'p2'],
  'j7': ['p1', 'p2'],
  'j8': ['p1', 'p2']}}

In [14]:
def eval_policy(model=model, env=env, n_eval_episodes=10):
    episode_rewards = []
    for _ in range(n_eval_episodes):
        obs = env.reset()
        episode_reward = 0
        done = False
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, _ = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    mean_reward = sum(episode_rewards) / n_eval_episodes
    std_reward = np.std(episode_rewards)
    return mean_reward, std_reward

In [ ]:
eval_policy()